In [1]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



l'objectif est de predire l'appentence des clients a des transport lowcoast.
Pour cela, nous utiliserons la librairie Ml de spark

perimetre: représente les identifaints des clients accessible à l'étude.
histo_client: represente l'historique des données clients sur une période donnée
histo_train: represente l'historique des données de commandes trains.
histo_lowcost: represente l'historique des données de client lowcost (défini avec le métier).
visites: représente l'historique des données de navigation des clients sur le site.

1 - lire les fichiers de données
2 - identifier les variables continues et transformer leurs modalités en double.
3 - joindre les differentes sources de données en se basant sur les données du périmètre (tous les individus du périmèetre devront apparaitre dans la jointure avec des valeurs NULL si nécessaire pour les colonnes en provenance d'autres sources).


1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [6]:
perimetre = spark_session.read.csv("data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("data_clients/sample_visites.csv", header=True)

In [7]:
## ecrire une fonction pour transformer les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float

In [8]:

client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast('double'))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


faire une jointure entre les informations des différentes tables.
NB: on conservera tous les clients de la table perimetre.
    En effet, ce sont les cleints qu'on souhaite scorer

combien a t'on de features quatitatives, qualitatives 

quelles sont les differentes modalites de la feature LBL_STATUT_CLT

quelles sont les features avec valeurs manquantes
remplacer les valeurs manquantes par -1 pour toutes les features

In [10]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds
#df = input_df(df)

Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost

In [11]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

##### features engineering et modélisation

In [13]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


#data, dff = preprocessed_df(df)

prelever un sample de data pour notre modelisation

#### Logistic regression

quelle est le label est renseigne pour la modelisation?

In [14]:

lr = LogisticRegression(labelCol="", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)


Ajuster le modele de regression logistique et calculer les coefficients de notre modele.

Evaluer votre modele courbe roc, precision, rappel, etc....

predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage

##### Random Forest

In [17]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

#model_rf = classifier.fit(data1)

evaluer les performance de notre modele

In [18]:
# une autre technique pour evaluer le modele

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

Evaluer soit meme le score en calculant le nombre de VP, FP, VN et FN
on calculera alors le score qui VP+VN/VP+VN+FP+FN
nb: la prediction est automatiquement creee dans le data set et correspond à la colonne prediction

In [56]:
calculer egalement le rappel et la precision

(1.0, 0.6846384667505566, 1.0, 0.9699609990872127)

In [19]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

Row(raw=DenseVector([0.9, 0.1]), label=0.0)

#### how to create a spark data frame

In [21]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25, "Girl"),
     (datetime.date(2018,2,3), 'Jalfaizy',22, "Girl"),
     (datetime.date(2018,1,5), 'saurabh',20, "Girl"),
     (datetime.date(2018,1,12), 'Bala',26, "Girl"),
     (datetime.date(2018,7,9), 'Jules',19, "Girl"),
     (datetime.date(2018,3,18), 'Arild',43, "Boy"),
     (datetime.date(2018,1,5), 'sarah',20, "Boy"),
     (datetime.date(2018,8,12), 'Boly',33, "Boy"),
     (datetime.date(2018,4,6), 'Anita',35, "Boy"),
     (datetime.date(2018,12,6), 'Jules',22, "Boy"),
     (datetime.date(2018,7,24), 'Soul',20, "Girl"),
     (datetime.date(2018,6,17), 'Gral',54, "Girl"),
     (datetime.date(2018,9,7), 'Apoh',18, "Girl"),
     (datetime.date(2018,10,4), 'Dony',32, "Girl"),
     (datetime.date(2018,2,5), 'Tanoh',31, "Girl"),
     (datetime.date(2018,11,12), 'Issouf',27, "Boy"),
     (datetime.date(2018,10,3), 'Bilé',29, "Boy"),
     (datetime.date(2018,5,3), 'Gagnon',20, "Boy"),
     (datetime.date(2018,3,5), 'Papiss',28, "Boy"),
     (datetime.date(2018,2,12), 'Kravitz',34, "Boy"),
     (datetime.date(2018,5,9), 'Mouli',35, "Girl"),
     (datetime.date(2018,8,3), 'Jacques',27, "Girl"),
     (datetime.date(2018,12,5), 'soum',22, "Girl"),
     (datetime.date(2018,4,12), 'MBra',36, "Girl")]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]), sex=x[3]))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name,sex
0,25,2018-01-03,Ankit,Girl
1,22,2018-02-03,Jalfaizy,Girl
2,20,2018-01-05,saurabh,Girl
3,26,2018-01-12,Bala,Girl
4,19,2018-07-09,Jules,Girl
5,43,2018-03-18,Arild,Boy
6,20,2018-01-05,sarah,Boy
7,33,2018-08-12,Boly,Boy
8,35,2018-04-06,Anita,Boy
9,22,2018-12-06,Jules,Boy


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.

In [22]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

,age,date,name,sex,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,Girl,2019-10-30,665
1,22,2018-02-03,Jalfaizy,Girl,2019-10-30,634
2,20,2018-01-05,saurabh,Girl,2019-10-30,663
3,26,2018-01-12,Bala,Girl,2019-10-30,656
4,19,2018-07-09,Jules,Girl,2019-10-30,478
5,43,2018-03-18,Arild,Boy,2019-10-30,591
6,20,2018-01-05,sarah,Boy,2019-10-30,663
7,33,2018-08-12,Boly,Boy,2019-10-30,444
8,35,2018-04-06,Anita,Boy,2019-10-30,572
9,22,2018-12-06,Jules,Boy,2019-10-30,328
